In [89]:
import pandas as pd
import numpy as np
from io import StringIO

In [90]:
csvdata = StringIO('''customer,date,a,b,c
123,01.01.17,1,0
234,02.01.17,0,0
345,01.01.17,0,0
456,04.01.17,1,1
567,03.01.17,1,0
678,08.01.17,1,1
789,13.01.17,0,1
890,05.01.17,1,0
123,02.01.17,1,0
234,03.01.17,0,0
345,02.01.17,0,0
456,05.01.17,1,1
567,04.01.17,1,0
678,10.01.17,1,1
789,18.01.17,0,1
890,15.01.17,1,0
123,10.01.17,1,0
234,20.01.17,0,0
345,15.01.17,0,0
456,24.01.17,1,1
567,30.01.17,1,0
678,18.01.17,1,1
789,23.01.17,0,1
890,25.01.17,1,0
123,1.2.17,1,0
234,2.2.17,0,0
345,1.2.17,0,0
456,4.2.17,1,1
567,3.2.17,1,0
678,8.2.17,1,1
789,13.2.17,0,1
890,5.2.17,1,0
123,2.2.17,1,0
234,3.2.17,0,0
345,2.2.17,0,0
456,5.2.17,1,1
567,4.2.17,1,0
678,10.2.17,1,1
789,18.2.17,0,1
890,15.2.17,1,0
123,10.2.17,1,0
234,20.2.17,0,0
345,15.2.17,0,0
456,24.2.17,1,1
567,27.2.17,1,0
678,18.2.17,1,1
789,23.2.17,0,1
890,25.2.17,1,0
123,1.3.17,1,0
234,2.3.17,0,0
345,1.3.17,0,0
456,4.3.17,1,1
567,3.3.17,1,0
678,8.3.17,1,1
789,13.3.17,0,1
890,5.3.17,1,0
123,2.3.17,1,0
234,3.3.17,0,0
345,2.3.17,0,0
456,5.3.17,1,1
567,4.3.17,1,0
678,10.3.17,1,1
789,18.3.17,0,1
890,15.3.17,1,0
123,10.3.17,1,0
234,20.3.17,0,0
345,15.3.17,0,0
456,24.3.17,1,1
567,30.3.17,1,0
678,18.3.17,1,1
789,23.3.17,0,1
890,25.3.17,1,0
''')

In [91]:
alldays = pd.DatetimeIndex(start='1.1.17',end='31.12.17',freq='D')
alldays[0:10]

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08',
               '2017-01-09', '2017-01-10'],
              dtype='datetime64[ns]', freq='D')

In [92]:
customers = [123, 234, 345, 456, 567, 678, 789, 890]
customers

[123, 234, 345, 456, 567, 678, 789, 890]

In [93]:
multiindex = pd.MultiIndex.from_product([customers,alldays],names=['customer','date'])

In [94]:
# indexer object that facilitates working with multi indices
# see also https://pandas.pydata.org/pandas-docs/stable/advanced.html#using-slicers
idx = pd.IndexSlice
idx

In [95]:
df = pd.read_csv(csvdata,sep=',')
df.head()

,customer,date,a,b,c
0,123,01.01.17,1,0,NaN
1,234,02.01.17,0,0,NaN
2,345,01.01.17,0,0,NaN
3,456,04.01.17,1,1,NaN
4,567,03.01.17,1,0,NaN


In [96]:
df.date = df.date.apply(lambda x: pd.datetime.strptime(x,'%d.%m.%y'))
df.head()

,customer,date,a,b,c
0,123,2017-01-01,1,0,NaN
1,234,2017-01-02,0,0,NaN
2,345,2017-01-01,0,0,NaN
3,456,2017-01-04,1,1,NaN
4,567,2017-01-03,1,0,NaN


In [97]:
df = df.set_index(['customer', 'date']).reindex(multiindex).fillna(0)
df.head()

a    b    c
customer date                     
123      2017-01-01  1.0  0.0  0.0
         2017-01-02  1.0  0.0  0.0
         2017-01-03  0.0  0.0  0.0
         2017-01-04  0.0  0.0  0.0
         2017-01-05  0.0  0.0  0.0

In [98]:
# use coloumns a through c for determining whether customer was active that day or not
activity = df[['a','b','c']].apply(any, axis=1)

# not any is used to determine wether customer churned or not. Not any activity means churned.
def notany(x):
    return not any(x)

# use the activity series to determine for each day whether the customer churned in the following n days
def churnNday(activity, n):
    return activity.rolling(n).apply(notany).groupby(level='customer').shift(-n).astype(bool)

In [99]:
churn7d = churnNday(activity, 7)
churn7d.head()

customer  date      
123       2017-01-01    False
          2017-01-02     True
          2017-01-03    False
          2017-01-04    False
          2017-01-05    False
dtype: bool

In [100]:
df['churn'] = churnNday(activity, 1)

In [101]:
df['chrun7day'] = churnNday(activity, 7)

In [102]:
# show first 10 days of every customer to validate the churn variables
df.loc[idx[:,:'2017-1-10'],:]

a    b    c  churn  chrun7day
customer date                                       
123      2017-01-01  1.0  0.0  0.0  False      False
         2017-01-02  1.0  0.0  0.0   True       True
         2017-01-03  0.0  0.0  0.0   True      False
         2017-01-04  0.0  0.0  0.0   True      False
         2017-01-05  0.0  0.0  0.0   True      False
         2017-01-06  0.0  0.0  0.0   True      False
         2017-01-07  0.0  0.0  0.0   True      False
         2017-01-08  0.0  0.0  0.0   True      False
         2017-01-09  0.0  0.0  0.0  False      False
         2017-01-10  1.0  0.0  0.0   True       True
234      2017-01-01  0.0  0.0  0.0   True       True
         2017-01-02  0.0  0.0  0.0   True       True
         2017-01-03  0.0  0.0  0.0   True       True
         2017-01-04  0.0  0.0  0.0   True       True
         2017-01-05  0.0  0.0  0.0   True       True
         2017-01-06  0.0  0.0  0.0   True       True
         2017-01-07  0.0  0.0  0.0   True       True
         2017-01-08  0.0  0.0  0.0   True       True
         2017-01-09  0.0  0.0  0.0   True       True
         2017-01-10  0.0  0.0  0.0   True       True
345      2017-01-01  0.0  0.0  0.0   True       True
         2017-01-02  0.0  0.0  0.0   True       True
         2017-01-03  0.0  0.0  0.0   True       True
         2017-01-04  0.0  0.0  0.0   True       True
         2017-01-05  0.0  0.0  0.0   True       True
         2017-01-06  0.0  0.0  0.0   True       True
         2017-01-07  0.0  0.0  0.0   True       True
         2017-01-08  0.0  0.0  0.0   True       True
         2017-01-09  0.0  0.0  0.0   True       True
         2017-01-10  0.0  0.0  0.0   True       True
...                  ...  ...  ...    ...        ...
678      2017-01-01  0.0  0.0  0.0   True      False
         2017-01-02  0.0  0.0  0.0   True      False
         2017-01-03  0.0  0.0  0.0   True      False
         2017-01-04  0.0  0.0  0.0   True      False
         2017-01-05  0.0  0.0  0.0   True      False
         2017-01-06  0.0  0.0  0.0   True      False
         2017-01-07  0.0  0.0  0.0  False      False
         2017-01-08  1.0  1.0  0.0   True      False
         2017-01-09  0.0  0.0  0.0  False      False
         2017-01-10  1.0  1.0  0.0   True       True
789      2017-01-01  0.0  0.0  0.0   True       True
         2017-01-02  0.0  0.0  0.0   True       True
         2017-01-03  0.0  0.0  0.0   True       True
         2017-01-04  0.0  0.0  0.0   True       True
         2017-01-05  0.0  0.0  0.0   True       True
         2017-01-06  0.0  0.0  0.0   True      False
         2017-01-07  0.0  0.0  0.0   True      False
         2017-01-08  0.0  0.0  0.0   True      False
         2017-01-09  0.0  0.0  0.0   True      False
         2017-01-10  0.0  0.0  0.0   True      False
890      2017-01-01  0.0  0.0  0.0   True      False
         2017-01-02  0.0  0.0  0.0   True      False
         2017-01-03  0.0  0.0  0.0   True      False
         2017-01-04  0.0  0.0  0.0  False      False
         2017-01-05  1.0  0.0  0.0   True       True
         2017-01-06  0.0  0.0  0.0   True       True
         2017-01-07  0.0  0.0  0.0   True       True
         2017-01-08  0.0  0.0  0.0   True      False
         2017-01-09  0.0  0.0  0.0   True      False
         2017-01-10  0.0  0.0  0.0   True      False

[80 rows x 5 columns]